In [ ]:
import csv
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
from bs4 import BeautifulSoup  # để loại bỏ các thẻ in đậm, nghiêng

subject_files = {
    "Văn": "output/van.csv",
#     "Sử": "output/lich_su.csv",
#     "Địa": "output/dia_ly.csv",
#     "Anh văn": "output/anhvan.csv"
}


: 

In [ ]:
# Mở trình duyệt
driver = webdriver.Edge()

In [ ]:
for subject, filepath in subject_files.items():
    print(f"\nĐANG XỬ LÝ MÔN: {subject}")
    results = []
    #lấy sl câu hỏi
    count = 0 

    with open(filepath, mode='r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        
        for row in reader:
            if count >= 50:
                break
    
            title = row["Title"]
            link = row["Link"]
            
            print(f"\n🔗 Truy cập: {title}")
            driver.get(link)
            time.sleep(2)  # đợi trang load

            questions = driver.find_elements(By.CSS_SELECTOR, "div.quiz-answer-item")
            for  q in questions:
                try:
                    # Câu hỏi (HTML -> text sạch)
                    question_html = q.find_element(By.CSS_SELECTOR, "div.title-question").get_attribute("innerHTML")
                    question_text = BeautifulSoup(question_html, "html.parser").get_text(separator=" ", strip=True)

                    # Đáp án
                    answer_elems = q.find_elements(By.CSS_SELECTOR, "div.answers label")
                    answers = [ans.text.strip() for ans in answer_elems if ans.text.strip()]
                    
                    # Lấy phần tử chứa giải thích và đáp án
                    reason_elem = q.find_element(By.CLASS_NAME, "reason")
                    reason_html = reason_elem.get_attribute("innerHTML")

                    import re
                    soup = BeautifulSoup(reason_html, "html.parser")
                    text = soup.get_text(separator="\n", strip=True) 
                    match = re.search(r"Đáp án(?: cần chọn là)?[:：]\s*([A-D])", text, re.IGNORECASE)
                    correct_answer = match.group(1).upper() if match else None
                    
                    if correct_answer:
                        explanation_text = re.sub(r"Đáp án(?: cần chọn là)?[:：]?\s*"+re.escape(correct_answer), "", text, flags=re.IGNORECASE).strip()
                    else:
                        explanation_text = text.strip()
                    cleaned_explanation = " ".join(explanation_text.split())
                    explanation = cleaned_explanation if cleaned_explanation and not cleaned_explanation.startswith("Đáp án") else None

                    result_item = {
                        "question": question_text,
                        "answers": answers,
                        "correct_answer": correct_answer
                    }

                    if explanation:
                        result_item["explanation"] = explanation

                    results.append(result_item)
                    count += 1
                    # #In kết quả
                    # print(f"Câu hỏi: {question_text}")
                    # for ans in answers:
                    #     print(f"   - {ans}")
                    # print("📘 Full text:", text)
                except:
                    continue

 # 💾 Lưu kết quả vào file JSON
    output_path = f"output/{subject.lower()}.json"
    with open(output_path, "w", encoding="utf-8") as out_file:
        json.dump(results, out_file, ensure_ascii=False, indent=2)
    print(f"\n✅ Đã lưu {len(results)} câu hỏi vào: {output_path}")

# Đóng trình duyệt
driver.quit()
                

